# HackOn(Data)
## Competition Challenge

Welcome to our project. We have decided to do a simple Forecasting on the salesRank, more precesilly, 

** The problem:** Knowing the first set of reviews of a product forecast the salesRank of it. 

** Hypothesis:** The reviews are an indicative of how well a product is selling and contain enough information for a forecast.

** Assumptions:** 

- We ommit that the data is seasonal.
- By product we mean the combination of product and vendors, that is represented by the asin.
- What else?

** NOTE: ** This notebook is intended to show the overall ideas and, in order to make it easier to read, we have sacrificed speed. In the last section when talking about scalability we dwell into these details.

# Part 0 - Loading the tools

We load the tools we will be using during this project.

In [1]:
import pandas as pd
import numpy as np
import fasttext
import gzip

from sklearn.preprocessing import MultiLabelBinarizer #To be used with the categories feature

%matplotlib inline  

# Part 1- Cleaning and preparing The Data

In this section we load the data, clean it, and prepare it for feature building. The data consists on

### **Review Data**

This dataset contains product reviews and metadata from Amazon, including 142.8 million reviews spanning May 1996 - July 2014.

This dataset includes reviews (ratings, text, helpfulness votes), product metadata (descriptions, category information, price, brand, and image features), and links (also viewed/also bought graphs).

### **Q&A Data**

This dataset contains Questions and Answers data from Amazon, totaling around 1.4 million answered questions.

This dataset can be combined with Amazon product review data, by matching ASINs in the Q/A dataset with ASINs in the review data. The review data also includes product metadata (product titles etc.).

### **Credits:**

-  **R. He, J. McAuley**. Modeling the visual evolution of fashion trends with one-class collaborative filtering. WWW, 2016 J. 
- ** McAuley, C. Targett, J. Shi, A. van den Hengel **. A Image-based recommendations on styles and substitutes. SIGIR, 2015


We start with the scripts provided at [Julian McAuley's website](http://jmcauley.ucsd.edu/data/amazon/).

In [2]:
def parse(path):
  g = gzip.open(path, 'rb')
  for l in g:
    yield eval(l)

def getDF(path):
  i = 0
  df = {}
  for d in parse(path):
    df[i] = d
    i += 1
  return pd.DataFrame.from_dict(df, orient='index')

and use these methods to load the data

In [3]:
df_reviews = getDF('./data/reviews_Video_Games_5.json.gz')
df_meta=getDF('./data/meta_Video_Games.json.gz')
df_qa=getDF('./data/qa_Video_Games.json.gz')

We now study the different features.


## Features from the meta file

### SalesRank

Note that the dataframe that contains the salesRank is df_meta. In this data there are two possible cases where the values described are not helpful. The first one is a NaN value, and we can use is null method to deal with this, the second one is the case of a dictionary no containing the relevant key. We use a helper function to help us with this.

In [4]:
RELEVANT_KEY='Video Games'
def helper(dictionary):
    try:
        return dictionary[RELEVANT_KEY]
    except:
        return float('NaN')

In [5]:
df_meta['salesRank']=df_meta.salesRank.map(lambda x: helper(x))

We can now count how many entries don't have the desired rank 
data.

In [6]:
df_meta.salesRank[(df_meta.salesRank.isnull())].shape[0]

5675

and proceed to remove these rows 

In [7]:
df_meta=df_meta[ df_meta.salesRank.notnull()]

We can do a sanity check 

In [8]:
df_meta.salesRank[(df_meta.salesRank.isnull())].shape[0]

0

### imUrl

This features contains the web address to an image. As ee won't be using the images, so we drop this column.

In [9]:
df_meta=df_meta.drop('imUrl',axis=1)

### Title and brand

From the fact that the percentage of articles with title or brands are

In [10]:
print("Only %.2f%% of the products have title."%(df_meta.title.count()/df_meta.asin.count()*100))
print("Only %.2f%% of the products have brand."%(df_meta.brand.count()/df_meta.asin.count()*100))

Only 0.22% of the products have title.
Only 0.11% of the products have brand.


We can deduce that this features are not relevant, and so we drop them as well.

In [11]:
df_meta=df_meta.drop(['title','brand'],axis=1)

### Price

The feature price is definitely relevant for the  forecasting, now note that 

In [12]:
print("%.2f%% of the products have a price."%(df_meta.price.count()*100/df_meta.asin.count()))

88.25% of the products have a price.


as this is a relevant feature we remove the rows with missing data.

In [13]:
df_meta=df_meta[df_meta.price.notnull()]

Another sanity check.

In [14]:
print("%.2f%% of the products have a price."%(df_meta.price.count()*100/df_meta.asin.count()))

100.00% of the products have a price.


After this cleaning 

In [15]:
print("There are %d products left"%df_meta.asin.count())

There are 39959 products left


### Categories

The categories feature comes as a list of lists, let's first find out how many different categories are there.

In [16]:
categories=set()
for list_cats in df_meta.categories:
    for list_cat in list_cats:
        categories=categories.union(set(list_cat))
categories=list(categories)
print("There are %d categories"%len(categories))

There are 334 categories


we use these as a categorial variable, first we create a unique list of categories for each product

In [17]:
def helper2(list_cats):
    cats=set()
    for list_cat in list_cats:
        cats=cats.union(set(list_cat))
    return list(cats)

In [18]:
df_meta['categories']=df_meta.categories.apply(helper2)

and then we use the [MultiLabelBinarizer](http://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.MultiLabelBinarizer.html) from sklearn to do the encodding. 

In [19]:
mlb=MultiLabelBinarizer()
mlb.fit(df_meta.categories)
categories_df=pd.DataFrame(mlb.transform(df_meta.categories),columns=list(mlb.classes_))
df_meta.reset_index(drop=True,inplace=True)
df_meta=df_meta.merge(categories_df,left_index =True,right_index =True)

and finally we drop the categories column.

In [20]:
df_meta=df_meta.drop('categories',axis=1)

### related and decription

(ARE WE USING THIS?, RELATED MAY GIVE NICE INFO, BUT DESCRIPTION SEEMS USELESS)

We won't be using this either, so we remove them.

In [21]:
df_meta=df_meta.drop(['related','description'],axis=1)

## Features from the reviews file

### reviewText

This feature containing the review will be used for sentiment analysis. We come back to this when building features. For now, we just get rid of the data with empty reviews.

In [22]:
print("There are %d empty reviews."%df_reviews[df_reviews.reviewText==""].shape[0])
df_reviews=df_reviews[df_reviews.reviewText!=""]

There are 44 empty reviews.


### summary

We won't be using this feature, so we drop it

In [23]:
df_reviews=df_reviews.drop('summary',axis=1)

### reviewTime

This feature is redundant since we have unixReviewTime, so we also get rid of this

In [24]:
df_reviews=df_reviews.drop('reviewTime',axis=1)

### ReviewerName
This is also redundant, since we have the reviewerID, we drop it

In [25]:
df_reviews=df_reviews.drop('reviewerName',axis=1)

# Part 2 - Feature engineering

In this part we build our features, this will be done in the following order.


- Use sentiment analysis to give a sentiment score to the reviews. (Reason: a score of 3 for two people may mean different things the sentiment wil help correct this)

- ReviewerID features: will help us gain confidence in the review.

- helpful: Increases the confidence.

- ReviewTime: To create a time series for the (cummulative) number of reviews.

- Categories: Create category 'Other' and move everything that is not in the top 20 categories to the 'Other' category.


### Sentiment Analysis on reviews

We use fasttext from facebook [CITE] to create an score. For this we need to save the data we need to a text file, since that is the signature of the training method of fasttext.

In [26]:
df_sentiment=df_reviews.loc[:,('reviewText','overall')]
df_sentiment['overall']=df_sentiment.overall.apply(lambda x: '__label__'+str(int(x)))
df_sentiment.to_csv(r'./data_/data_for_sentiment.txt', header=None, index=None, sep=' ', mode='a')

Now, that the data is prepared for procesing, we can use the classifier.

In [27]:
classifier = fasttext.supervised('./data_/data_for_sentiment.txt', 'model', label_prefix='__label__')

and do a simple, sanity check

In [28]:
texts=["this is an awesome","this sucks"]
labels = classifier.predict(texts)
print(labels)

[['5'], ['1']]


and we can add this feature to the reviews dataframe

In [29]:
df_reviews['sentimentScore']=df_reviews.reviewText.apply(
    lambda x:int(classifier.predict([x])[0][0]))

as that was our use of the textReview feature we can get rid of it.

In [30]:
df_reviews=df_reviews.drop('reviewText',axis=1)

In [31]:
df_reviews.head()

,asin,overall,reviewerID,unixReviewTime,helpful,sentimentScore
0,0700099867,1.0,A2HD75EMZR8QLN,1341792000,"[8, 12]",1
1,0700099867,4.0,A3UR8NLLY1ZHCX,1372550400,"[0, 0]",3
2,0700099867,1.0,A1INA0F5CWW3J4,1403913600,"[0, 0]",5
3,0700099867,3.0,A1DLMTOTHQ4AST,1315958400,"[7, 10]",1
4,0700099867,4.0,A361M14PU2GUEG,1308009600,"[2, 2]",5


### ReviewerID

We want to measure the quality of a review, so far we only have a values for this, the helpful feature. We add a new feature that comes from the reviewerID where we say that a review has better quality the larger the number of reviews the corresponding revierID has. We start by counting frequencies of the reviewerID. 

In [48]:
value_counts_reviewerID=df_reviews.reviewerID.value_counts().to_dict()

and add the feature reviewerQuality

In [49]:
df_reviews['reviewerQuality']=df_reviews.reviewerID.apply(lambda x: value_counts_reviewerID[x])

as we won't be using the reviewerID anymore, we get rid of this feature.

In [50]:
df_reviews=df_reviews.drop('reviewerID',axis=1)

## Data aggregation

Before we continue with the feature generation, we need to agregate the data by product. This will make the features to be a time series, where the time is controled by the UnixReviewTime.


In [55]:
df_products = df_reviews.groupby('asin').agg(lambda x:(list(x)))

### helpful

In order to incorporate the helpful score into the product, we create two variables defined below:

$$ \text{helpfulOverall} = \sum \text{overall}_i \cdot \frac{\text{helpful}_i[0]}{\text{helpful}_i[1]} $$ 

and 

$$ \text{helpfulSentiment} = \sum \text{sentiment_score}_i \cdot \frac{\text{helpful}_i[0]}{\text{helpful}_i[1]} $$ 
where the sums run over the $i$ such that helpful$_i$[1]$\neq 0$.

We need a helper function to incorporate these new features.

In [56]:
def helper_helpful(A,B):
    total=0
    for a,b in zip(A,B):
        b1,b2=b
        if b2!=0:
            total+=a*b1/b2
    return total

In [57]:
df_products['helpfulOverall']=df_products.apply(lambda row: helper_helpful(row['overall'],row['helpful']),axis=1)
df_products['helpfulSentiment']=df_products.apply(lambda row: helper_helpful(row['sentimentScore'],row['helpful']),axis=1)

and we can drop the helpful feature now.

In [59]:
df_products=df_products.drop(['helpful'],axis=1)

### reviewerQuality

As we want to introduced an invariant that captures the quality of the review, we can use this in the same way as with helpful to produce two more features.

$$ \text{qualityOverall} = \sum \text{overall}_i \cdot \text{reviewerQuality}_i $$ 

and 

$$ \text{qualitySentiment} = \sum \text{sentiment}_i \cdot \text{reviewerQuality}_i $$ 

as before, we need a helper function

In [61]:
def helper_quality(A,B):
    total=0
    for a,b in zip(A,B):
        total+=a*b
    return total

In [63]:
df_products['qualityOverall']=df_products.apply(lambda row: helper_quality(row['overall'],row['reviewerQuality']),axis=1)
df_products['qualitySentiment']=df_products.apply(lambda row: helper_quality(row['sentimentScore'],row['reviewerQuality']),axis=1)

and we are ready to drop more features. 

In [66]:
df_products=df_products.drop(['overall','sentimentScore','reviewerQuality'],axis=1)

**REMARK:** The reader should note that there is a faster way to compute the last two features, readly starting from the df_reviews dataframe building the individual multiplications and then aggregating. We opted for the (significantly) slower way to make the notebook easier to read.

### The hotness feature



In [67]:
df_products.head()

,unixReviewTime,helpfulOverall,helpfulSentiment,qualityOverall,qualitySentiment
asin,,,,,
0700099867,"[1341792000, 1372550400, 1403913600, 131595840...",32.431750,35.031750,1212.0,1276
6050036071,"[1334016000, 1258588800, 1396656000, 129755520...",12.000000,8.000000,274.0,216
7100027950,"[1333497600, 1257724800, 1392940800, 123223680...",15.041667,13.291667,576.0,550
7293000936,"[1360886400, 1315180800, 1380672000, 132062400...",5.000000,5.000000,235.0,251
8176503290,"[1335657600, 1363305600, 1363564800, 132304320...",4.000000,6.000000,178.0,206
